In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
df = pd.read_csv('/kaggle/input/coronavirus-2019ncov/covid-19-all.csv')

In [ ]:
df.head()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df[['Confirmed','Recovered','Deaths']] = df[['Confirmed','Recovered','Deaths']].fillna(0).astype(int)
df['Still Infected'] = df['Confirmed'] - df['Recovered'] - df['Deaths']
df["Country/Region"].replace({"Mainland China": "China"}, inplace=True)
countries_affected = df['Country/Region'].unique().tolist()
print("\nTotal countries effected by Corona virus: ",len(countries_affected))
df.head()

In [ ]:
df_list= df['Country/Region'].unique().tolist()
print ('[%s]' % ', '.join(map(str, df_list)))

In [ ]:
del df['Province/State']
df.head()

In [ ]:
aggregation_functions = {'Confirmed': 'sum', 'Recovered': 'sum', 'Deaths': 'sum', 'Still Infected': 'sum', }
df_new = df.groupby(['Country/Region', 'Date']).aggregate(aggregation_functions)
df_new.reset_index()
df_new.head()

In [ ]:
del df['Latitude']
del df['Longitude']
df.head()

In [ ]:
c_df = df[df['Country/Region'] == 'India']
c_df.head()

In [ ]:
del c_df['Country/Region']
c_df = c_df.sort_values(["Date"], ascending = (True)).reset_index()
c_df.head()

In [ ]:
c_df = c_df.groupby(['Date'])[['Confirmed','Recovered','Deaths','Still Infected']].sum().reset_index()
c_df.head()

In [ ]:
ax1 = c_df.plot.scatter(x='Date', y='Confirmed', style='b')
c_df.plot.line(x='Date', y='Confirmed', ax=ax1, style='b')
c_df.plot.line(x='Date', y='Recovered', ax=ax1, style='g')
c_df.plot.line(x='Date', y='Deaths', ax=ax1, style='r')

In [ ]:
df_list= df['Country/Region'].unique().tolist()
# country_list = '[%s]' % ', '.join(map(str, df_list))
# print(country_list)
for i in df_list:
    c_df = df[df['Country/Region'] == i]
    del c_df['Country/Region']
    c_df = c_df.sort_values(["Date"], ascending = (True)).reset_index()
    c_df = c_df.groupby(['Date'])[['Confirmed','Recovered','Deaths','Still Infected']].sum().reset_index()
    ax1 = c_df.plot.scatter(x='Date', y='Confirmed', style='b')
    c_df.plot.line(x='Date', y='Confirmed', ax=ax1, style='b')
    c_df.plot.line(x='Date', y='Recovered', ax=ax1, style='g')
    c_df.plot.line(x='Date', y='Deaths', ax=ax1, style='r')
    
    

In [ ]:
recent_date = df['Date'].max()
latest_entry = df[df['Date'] >= recent_date]
df_stats = df.groupby(['Date','Country/Region'])[['Confirmed','Recovered','Deaths','Still Infected']].sum().reset_index()
df_stats.head()

In [ ]:
df_all_agg = df.drop('Date',1)
df_all_agg.head()

In [ ]:
df_all_agg = df_all_agg.groupby(['Country/Region'])[['Confirmed','Recovered','Deaths','Still Infected']].sum().reset_index()
df_all_agg.head()

In [ ]:
latest_entry.head()

In [ ]:
latest_entry.shape

In [ ]:
latest_entry = latest_entry.groupby(['Country/Region'])[['Confirmed','Recovered','Deaths','Still Infected']].sum().reset_index()
latest_entry.head()

In [ ]:
latest_entry['Death/Case'] = (latest_entry['Deaths']*100)/latest_entry['Confirmed']

In [ ]:
latest_entry.head()

In [ ]:
latest_entry.shape

In [ ]:
latest_entry['Recovery/Case'] = (latest_entry['Recovered']*100)/latest_entry['Confirmed']
latest_entry.head()

In [ ]:
latest_entry['Mortality'] = (latest_entry['Deaths']*100)/latest_entry['Recovered']
latest_entry.head()

In [ ]:
q1 = latest_entry['Confirmed'].quantile(0.25)
q3 = latest_entry['Confirmed'].quantile(0.75)
iqr = q3 - q1
fence_low = q1 - 1.5 * iqr
fence_high = q3 + 1.5 * iqr
cleaned_data = latest_entry.loc[(latest_entry['Confirmed'] > fence_low) & (latest_entry['Confirmed'] < fence_high)]
cleaned_data.head()

In [ ]:
latest_entry = latest_entry.sort_values(["Still Infected", "Deaths", "Recovery/Case", "Mortality"], ascending = (True, True, False, True)).reset_index()

latest_entry.head()

In [ ]:
latest_entry.tail()

In [ ]:
latest_entry = latest_entry[latest_entry['Mortality'] > 0]
latest_entry.tail()

In [ ]:
latest_entry.head()

In [ ]:
latest_entry.shape

In [ ]:
t = latest_entry[latest_entry['Confirmed'] > 1500]
t.head()

In [ ]:
t.tail()

# Visualization

**============================================**

In [ ]:
import folium

In [ ]:
import requests
import json
url = '/kaggle/input/folium-world-map-data/general_map_data.json'
f = open(url,)
data = json.load(f)

In [ ]:
map_poly_data = {}
country_list = []
for i in data['features']:
    country = i['properties']['name']
    polygon = i['geometry']['coordinates']
    map_poly_data[country] = polygon
    country_list.append(country)
print(len(country_list))

In [ ]:
print(country_list)

In [ ]:
df_c_l = t['Country/Region'].tolist()
sorted(df_c_l)

In [ ]:
sorted(country_list)
country_list

In [ ]:
aug_list = df_c_l + country_list
c_set = set(aug_list)
c_list = sorted(list(c_set))

In [ ]:
c_list

In [ ]:
miss = set(df_c_l) - set(country_list)
miss

In [ ]:
t.replace('Burma', "Myanmar", inplace = True)

fields = ['Confirmed','Recovered','Deaths','Still Infected','Death/Case','Recovery/Case','Mortality']
for i in fields:
    t[t['Country/Region']=='Congo (Brazzaville)'][i] = t[t['Country/Region']=='Congo (Brazzaville)'][i] + t[t['Country/Region']=='Congo (Kinshasa)'][i]
t.replace("Congo (Brazzaville)", "Democratic Republic of the Congo", inplace = True)
t = t.drop(t[t['Country/Region']=='Congo (Kinshasa)'].index)
t.replace("US","United States of America" , inplace = True)
t.replace("West Bank and Gaza", "West Bank", inplace = True)
t.replace("Serbia", "Republic of Serbia", inplace = True)
t.replace("North Macedonia", "Macedonia", inplace = True)
t.replace("Guinea-Bissau", "Guinea Bissau", inplace = True)


In [ ]:
x = t['Mortality'].nlargest(3).tolist()
max_i = x[0]
max_val = x[2]
t.loc[t['Mortality'] == max_i, ['Mortality']] = max_val

In [ ]:
t.loc[t['Mortality'] > 100, ['Mortality']] = 100

In [ ]:
miss = set(country_list) - set(df_c_l)
miss

In [ ]:
t.tail()

In [ ]:
m = folium.Map()
folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=data,
    name='Death/Case COVID-19',
    data=t,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['Country/Region', 'Death/Case'],
    key_on='feature.properties.name',
    fill_color='PuRd',
    nan_fill_color='white'
).add_to(m)

m

In [ ]:
m = folium.Map()
folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=data,
    name='Total Case COVID-19',
    data=t,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['Country/Region', 'Confirmed'],
    key_on='feature.properties.name',
    fill_color='PuRd',
    nan_fill_color='white'
).add_to(m)

m

In [ ]:
m = folium.Map()
folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=data,
    name='Total Deaths COVID-19',
    data=t,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['Country/Region', 'Deaths'],
    key_on='feature.properties.name',
    fill_color='PuRd',
    nan_fill_color='white'
).add_to(m)

m

In [ ]:
m = folium.Map()
folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=data,
    name='Recovery/Case COVID-19',
    data=t,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['Country/Region', 'Recovery/Case'],
    key_on='feature.properties.name',
    fill_color='PuRd',
    nan_fill_color='white'
).add_to(m)

m

In [ ]:
m = folium.Map()
folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=data,
    name='Mortality COVID-19',
    data=t,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['Country/Region', 'Mortality'],
    key_on='feature.properties.name',
    fill_color='PuRd',
    nan_fill_color='white'
).add_to(m)

m